In [1]:
import functions as Q
import requests
import numpy as np
from scipy.linalg import expm
from skopt import BayesSearchCV
from skopt.space import Integer
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from scipy.signal import convolve

def lobster_data(data1, data2):
    # Extract time from the first column of data1
    Time = data1[:, 1]

   # Assuming data2 is a NumPy array
    num_rows, num_cols = data2.shape
    num_levels = num_cols // 4

    # Reshape data2 to match the desired structure
    data_reshaped = data2.reshape((num_rows, num_levels, 4))

    # Extract MOBid, bid_volume, MOAsk, and ask_volume from reshaped data
    MOAsk = data_reshaped[:, :, 0]  # Ask Price
    ask_volume = data_reshaped[:, :, 1]  # Ask Size 
    MOBid = data_reshaped[:, :, 2]  # Bid Price
    bid_volume = data_reshaped[:, :, 3]  # Bid Size

    return MOBid, bid_volume, MOAsk, ask_volume, Time
# Replace 'your_file.csv' with the actual path to your CSV file
file_path1 = 'C:\\Users\\Maxparizot\\Documents\\GitHub\\TradeonQ\\LOB3Example - Matlab example\\LOBSupportingFiles\\MSFT_2012-06-21_34200000_57600000_message_10.csv'
file_path2 = 'C:\\Users\\Maxparizot\\Documents\\GitHub\\TradeonQ\\LOB3Example - Matlab example\\LOBSupportingFiles\\MSFT_2012-06-21_34200000_57600000_orderbook_10.csv'

# Read the CSV file into a DataFrame
data1 = np.loadtxt(file_path1,delimiter=',')
data2= np.loadtxt(file_path2,delimiter=',')

MOBid, bid_volume, MOAsk, ask_volume, Time = lobster_data(data1, data2)

# Calculate midprice for each entry in MoBid and MoAsk
S = np.array([Q.midprice(MOBid[price, 0], MOAsk[price, 0]) for price in range(MOBid.shape[0])])

# find Imbalance index
I = np.array([Q.imbalance_ratio(bid_volume[time, :], ask_volume[time, :]) for time in range(bid_volume.shape[0])])


# Set your reference time (midnight)
reference_time = datetime(2012, 6, 21, 9, 30, 0)
t = [reference_time + timedelta(seconds=duration) for duration in Time]

# Creating a DataFrame in Python using Pandas
Data = pd.DataFrame({'t': t, 'S': S, 'I': I, 'MOBid': MOBid[:,0], 'MOAsk': MOAsk[:,0]})
# 't' is the timestamp, set it as the index

#separate data into training and validating data
bp = round(0.80 * len(t))  # Use 80% of data for training
TData = Data.iloc[:bp, :]  # Training data
VData = Data.iloc[bp:, :]  # Validation data
VData = VData.reset_index(drop=True)  # Reset index of validation data

In [2]:
#validate makeQ and trade on Q
n=6
N=36
# Compare Qs
rho, DS = Q.get_states(TData, n, N)

QT= Q.make_Q(TData, n, N)

cash=Q.trade_on_Q(VData,QT,n,N)
print(cash)
#QV, PCondQV, PQV, GQV, HQV, vQV, CQV, phiQV= Q.make_Q(VData, n, N)
#QTVDiff = QT - QV
#Inhomogeneity = (QT > 0.5) & (QV < 0.5) | (QT < 0.5) & (QV > 0.5)
#print("Inhomogeneity: ", Inhomogeneity)
# Map states to a composite state space 'phi'


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


6200.0


In [2]:
from skopt import BayesSearchCV

results=Q.optimize_trading(TData, VData)
print(results)

TypeError: BayesSearchCV.__init__() got an unexpected keyword argument 'acq_optimizer'